In [5]:
import numpy as np
import random

# oh so pretty!
def float_formatter(x):
    if x < 0.001 and x > 0:
        return "   >0" 
    elif x == 0:
        return "    0" 
    else:
        return "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

def _logsum(logx, logy):
    print("Test")
    print(logx)
    print(logy)
    if len(logx.shape) < 1:
        if np.isinf(logx) or np.isinf(logy):
            print("result")
            print(np.max([logx, logy]))
            return np.max([logx, logy])
        else:
            print("result")
            print(np.logaddexp(logx, logy))
            return np.logaddexp(logx, logy)
    else:
        b = np.any([np.isinf(logx), np.isinf(logy)], axis=1)
        result = np.zeros(shape=logx.shape[0])
        result[b] = np.maximum(logx[b], logy[b])
        result[not b] = np.logaddexp(logx[not b], logy[not b])
        print("result")
        print(result)
        return result

def write_fasta_file(annotation_file_to_save, annotation_string, genome_file=None):
    if (genome_file!=None):
        with open(genome_file) as gf:
            first_line = gf.readline()
            s = " "
            words = first_line.split(s)
            words[0] = words[0] + " gene annotation"
            first_line = s.join(words[:-1])
    else:
        first_line = "; gene annotation"
    with open(annotation_file_to_save, 'w+') as af:
        af.write(first_line + '\n')
        name = annotation_file_to_save.replace('/', '\\').split('\\')[-1].split('.fa')[0]
        af.write('>' + name + '\n')
        line_length = 60
        full_lines = len(annotation_string)//line_length
        for anot_line in range(full_lines):
            af.write(annotation_string[anot_line*line_length:(anot_line+1)*line_length] + '\n')
        af.write(annotation_string[(full_lines)*line_length:])
       
    
# eg
#write_fasta_file("data/annotation13.fa", "ABABABABABABABABABAAAAJHASJHASJJSdskfjklsfkjklj sdhjkøfhjkø ksdhjf jkjkhsdfjk hjksdf hjksdfjk jhksd fhjkdsfhjk AJJSJHASHJASJJHASJHASJHASJHSAJJHASJHASHJASJJHASJHASJJHASJH")

In [6]:
def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

genome = ''
annotation = ''
for i in range(4,5):
    genome += read_fasta_file('data/genome%d.fa' % i)['genome%d' % i]
    annotation += read_fasta_file('data/annotation%d.fa' % i)['annotation%d' % i]

In [7]:
#Here we load our model
npzfile = np.load('codon_hmm_excl5 - Copy.npz')
A = npzfile['A']
E = npzfile['E']
pi = npzfile['pi']
d = np.zeros((A.shape[0], 3))
d[0, 0] = 1
for i in range(1, d.shape[0]):
    d[i, 2] = 1
print(d)
print(d[:, 0])

# define all possible outcomes of all states:
outcomes_dict = {} # ohh gawd this is akward
outcomes = np.empty([68], dtype=object) # help me, im lost
i = 0
letters = 'ACGT'
for letter1 in letters:
    outcomes[i] = letter1
    outcomes_dict[letter1] = i
    i += 1
    for letter2 in letters:
        for letter3 in letters:
            outcomes[i] = letter1 + letter2 + letter3
            outcomes_dict[letter1 + letter2 + letter3] = i
            i += 1
            


state_codes=npzfile['state_codes']
state_desc=npzfile['state_desc']

q = 3*np.ones(len(state_desc), dtype=np.int32)
q[0] = 1
print(state_desc)
print(state_codes)

def l2i(l):
    if type(l) is str:
        if l == "" or len(l) == 2:
            return -1
        else:
            return outcomes_dict[l]
    else:
        return [l2i(l_) for l_ in l]
#i can either be a list of indices, or just an index
def i2l(i):
    if type(i) is int:
        return outcomes[i]
    else:
        return [outcomes[i_] for i_ in i]
def e_prop(E, s):
    result = list()
    for i in range(len(s)):
        result.append(E[i, s[i]])
    return result

[[1.000     0     0]
 [    0     0 1.000]
 [    0     0 1.000]
 [    0     0 1.000]
 [    0     0 1.000]
 [    0     0 1.000]
 [    0     0 1.000]]
[1.000     0     0     0     0     0     0]
['Noncoding' 'C-start' 'C-codon' 'C-stop' 'R-start' 'R-codon' 'R-stop']
['N' 'C' 'C' 'C' 'R' 'R' 'R']


In [10]:
#This is the Viterbi algorithm, where different states, can take input of different length.
#A, E, p and s is as above. q is a vector, signifying the length of inputs for each state.

s = genome

def logaddexplist(l):
    result = 0
    for element in l:
        result = np.logaddexp(result, element)
    return result


def viterbi_ext_gen(A, E, p, s, q, d):
    A = np.log(A)
    E = np.log(E)
    p = np.log(p)
    d = np.log(d)
    #Then we generate some result matrix, with -inf everywhere.
    result = np.log(np.zeros([A.shape[0], len(s)]))
    
    #First we make the base cases
    obs = l2i([s[0:i] for i in range(d.shape[1])])
    #first row
    result[:, 0] = p[:,0] + d[:, 0] + E[:, obs[0]]
    print(p[:,0])
    print(d[:, 0])
    print(E[:, obs[0]])
    print(result)
    print()
    #second row
    result[:, 1] = p[:,0] + d[:, 1] + np.sum(E[:, [obs[0], obs[1]]], axis=1)
    for i in range(A.shape[0]):
        for j in range(A.shape[0]):
            if j != i:
                result[i, 1] = _logsum(result[i, 1], result[j, 0] + A[i,j] + d[i, 1] + E[i, obs[1]])
                                  
    result[:, 2] = p[:,0] + d[:, 2] + np.sum(E[:, [obs[0], obs[1], obs[2]]], axis=1)
    print(result)
    print()
    for k in range(1, 3):
        for i in range(A.shape[0]):
            for j in range(A.shape[0]):
                if j != i:
                    e_ = 0
                    for l in range(3-k, 3):
                        e_ += E[i, obs[l]]
                    result[i, 2] = _logsum(result[i, 2], result[j, 3-k] + A[i,j] + d[i, 2] + e_)
    print(result)
    print()
    return result

def viterbi_ext_path(X, A, E, p, s, q):
    A = np.log(A)
    E = np.log(E)
    p = np.log(p)
    n = X.shape[1]-2
    j = len(s)-2
    path = -1*np.ones(n+1, dtype=np.int32)
    path[-2::] = 0
    for i in range(n, 0, -1):
        q_ = q[path[i]]
        s_ = s[j-q_:j+(3-q_)]
        p = E[path[i], l2i(s_)]
        p = p + X[:, i-1]
        p = p + A[:, path[i]]
        path[i-1] = np.argmax(p)
        j = j-q_
    return path

def viterbi_ext_annotation(genome, path):
    result = np.empty(len(path), dtype=object)
    result[path == 0] = 'N'
    result[path == 1] = 'CCC'
    result[path == 2] = 'CCC'
    result[path == 3] = 'CCC'
    result[path == 4] = 'RRR'
    result[path == 5] = 'RRR'
    result[path == 6] = 'RRR'
    print(result.shape)
    result = "".join(result)
    print(len(result))
    return result

X = viterbi_ext_gen(A, E, pi, s, q, d)
#print(X)
#path = viterbi_ext_path(X, A, E, pi, s, q)
#print(path)
#print((path!=0).sum())
#for i in range(path.shape[0]):
#    if path[i] == 1:
#        print(X[:, i-2:i+2])
#        break
#annotation_pred = viterbi_ext_annotation(s, path)
#print(len(genome))
#print(len(annotation_pred))
#print(annotation_pred)
#write_fasta_file('data/annotation_prediction_5.fa', annotation, 'data/genome5.fa')

C:\Users\Morten\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: divide by zero encountered in log
C:\Users\Morten\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: RuntimeWarning: divide by zero encountered in log
C:\Users\Morten\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: divide by zero encountered in log
C:\Users\Morten\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: RuntimeWarning: divide by zero encountered in log


[    0 -inf -inf -inf -inf -inf -inf]
[    0 -inf -inf -inf -inf -inf -inf]
[-inf -inf -3.415 -inf -inf -2.899 -inf]
[[-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]
 ..., 
 [-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]]

Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-inf
Test
-inf
-inf
result
-in

C:\Users\Morten\Anaconda3\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: divide by zero encountered in log


In [19]:
S = np.arange(4).reshape(2,2)
print(S)
print(S.prod(axis=1))

[[0 1]
 [2 3]]
[0 6]


In [45]:
#
# compare_anns.py <true> <pred>
#
# compares a predicted gene structure against the true gene structure and computes
# various statistics summarizing the quality of the prediction. The argument <true> is 
# the true gene structure in faste format, and <pred> is the predicted gene structure 
# in fasta format, e.g.
#
# > python compare_anns.py ./annotation1.fa ./pred1.fa
# > Only Cs (tp=728238, fp=0, tn=505177, fn=249):
# > Sn = 0.9997, Sp = 1.0000, CC = 0.9996, AC = 0.9996
# > Only Rs (tp=618777, fp=0, tn=505426, fn=0):
# > Sn = 1.0000, Sp = 1.0000, CC = 1.0000, AC = 1.0000
# > Both (tp=1347015, fp=0, tn=505177, fn=249):
# > Sn = 0.9998, Sp = 1.0000, CC = 0.9997, AC = 0.9997
#
# Christian Storm <cstorm@birc.au.dk>
#

import os
import sys
import string
import math

def read_ann(filename):
    lines = []
    for l in open(filename).readlines():
        if l[0] != ">" and l[0] != ';':
            lines.append(l.strip())
    return "".join(lines)

def count_c(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'C' or pred[i] == 'c':
            total = total + 1
            if true[i] == 'C' or true[i] == 'c':
                tp = tp + 1
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n' or true[i] == 'R' or true[i] == 'r':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def count_r(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'R' or pred[i] == 'r':
            total = total + 1
            if true[i] == 'R' or true[i] == 'r':
                tp = tp + 1
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n' or true[i] == 'C' or true[i] == 'c':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def count_cr(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'C' or pred[i] == 'c' or pred[i] == 'R' or pred[i] == 'r':
            total = total + 1
            if (pred[i] == 'C' or pred[i] == 'c') and (true[i] == 'C' or true[i] == 'c'):
                tp = tp + 1
            elif (pred[i] == 'R' or pred[i] == 'r') and (true[i] == 'R' or true[i] == 'r'):
                tp = tp + 1                
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def print_stats(tp, fp, tn, fn):
    sn = float(tp) / (tp + fn)
    sp = float(tp) / (tp + fp)
    cc = float((tp*tn - fp*fn)) / math.sqrt(float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    acp = 0.25 * (float(tp)/(tp+fn) + float(tp)/(tp+fp) + float(tn)/(tn+fp) + float(tn)/(tn+fn))
    ac = (acp - 0.5) * 2
    print("Sn = %.4f, Sp = %.4f, CC = %.4f, AC = %.4f" % (sn, sp, cc, ac))

def print_all(true, pred):
    (totalc, tp, fp, tn, fn) = count_c(true, pred)
    if totalc > 0:
        print("Only Cs (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)

    (totalr, tp, fp, tn, fn) = count_r(true, pred)
    if totalr > 0:
        print("Only Rs (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)

    (total, tp, fp, tn, fn) = count_cr(true, pred)
    if totalc > 0 and totalr > 0:
        print("Both (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)


# Read true annotation
true_ann = read_ann('data/annotation5.fa')

# Read predicted annotations
pred_ann = read_ann('data/annotation_prediction_5.fa')
# Check annoation length
error = 0
if len(true_ann) != len(pred_ann):
    print("ERROR: The lengths of two predictions are different")
    print("Expected %d, but found %d" % (len(true_ann), len(pred_ann)))
    sys.exit(1)    
    
# Print stats
print_all(true_ann, pred_ann)


ERROR: The lengths of two predictions are different
Expected 1796846, but found 2053385


SystemExit: 1

/home/morten/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
